# 06. 이미지 회전과 원근 변환

In [1]:
import cv2 as cv
import numpy as np
DOG_PATH = "../images/dog.jpg"
CARD_PATH = "../images/card.jpg"

## 6-1. 이미지 회전(rotate)
- ```cv2.rotate(img, rotateCode)```
- ```rotateCode```
    - ```cv2.ROTATE_90_CLOCKWISE``` : 시계방향 90도 회전
    - ```cv2.ROTATE_180``` : 180도 회전
    - ```cv2.ROTATE_90_COUNTERCLOCKWISE``` : 반시계방향 90도 회전

In [ ]:
img = cv.imread(DOG_PATH)

dst_90 = cv.rotate(img, cv.ROTATE_90_CLOCKWISE)
dst_180 = cv.rotate(img, cv.ROTATE_180)
dst_90_counter = cv.rotate(img, cv.ROTATE_90_COUNTERCLOCKWISE)

cv.imshow("original", img)
cv.imshow("90", dst_90)
cv.imshow("180", dst_180)
cv.imshow("90 counter", dst_90_counter)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

## 6-2. 이미지 회전(아핀 변환)
- 이미지를 이동, 회전, 크기 조정, 반사, 기울이기(뒤틀기)와 같은 변환을 수학적으로 처리하는 방법

In [ ]:
# 아핀 변환(math 모듈로 직접 구현)
import math

img = cv.imread(DOG_PATH)

rad = 45 * math.pi / 180
affine = np.array([[math.cos(rad), -math.sin(rad), 0], [math.sin(rad), math.cos(rad), 0]])
scale = (img.shape[1], img.shape[0])

dst = cv.warpAffine(img, affine, scale)

cv.imshow("img", img)
cv.imshow("rotate", dst)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
# 아핀 변환(getRotationMatrix2D 이용)

img = cv.imread(DOG_PATH)

center = (int(img.shape[1]/2), int(img.shape[0]/2))
scale = (img.shape[1], img.shape[0])
affine = cv.getRotationMatrix2D(center, 45, 1)

dst = cv.warpAffine(img, affine, scale)

cv.imshow("img", img)
cv.imshow("rotate", dst)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
# 트략바와 함께 사용

img = cv.imread(DOG_PATH)

name = "Rotation"
cv.namedWindow(name)

trackbar_name = "Angle"
cv.createTrackbar(trackbar_name, name, 0, 360, lambda x:x)

while True:
    angle = cv.getTrackbarPos(trackbar_name, name)
    center = (int(img.shape[1]/2), int(img.shape[0]/2))
    scale = (img.shape[1], img.shape[0])
    affine = cv.getRotationMatrix2D(center, angle, 0.5)
    dst = cv.warpAffine(img, affine, scale)

    cv.imshow(name, dst)

    if cv.waitKey(0) == ord("q"):
        break

cv.destroyAllWindows()
cv.waitKey(1)

## 6-3. 원근 변환
- 원근법의 원리를 이용해 이미지에서 특정 부분의 기하학적 왜곡을 수정하거나 시점을 변경하는 것
- ```cv2.getPerspectiveTransform(src, dst) -> mat```
- ```cv2.warpPerspective(img, mat, (width, height))```

In [ ]:
img = cv.imread(CARD_PATH)

width, height = 600, 350

# 원근을 변형할 지점 선택 : 시계 방향
src = np.array([[54,261], [981,128], [1213,560], [194,735]], dtype=np.float32)

# 결과물을 출력할 형태 : 시계 방향
dst = np.array([[0,0], [width,0], [width,height], [0,height]], dtype=np.float32)

# 원근을 변형하기 위한 Matrix 생성
mat = cv.getPerspectiveTransform(src, dst)

# img를 mat에 의해 변환
result = cv.warpPerspective(img, mat, (width, height))

cv.imshow("img", img)
cv.imshow("warped", result)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
# 실습1. 이미지 원근 변환하기

PRACTICE_PATH = "../images/book2.jpg"

img = cv.imread(PRACTICE_PATH)

width, height = 200, 300

src = np.array([[153,113], [345,48], [505,277], [280,387]], dtype=np.float32)
dst = np.array([[0,0], [width,0], [width,height], [0,height]], dtype=np.float32)
mat = cv.getPerspectiveTransform(src, dst)
result = cv.warpPerspective(img, mat, (width, height))

cv.imshow("img", img)
cv.imshow("warped", result)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
src = np.array([[153,113], [345,48], [505,277], [280,387]], dtype=np.float32)

int(src[1][1])

## 6-4. 마우스 이벤트
- ```cv2.setMouseCallback(winName, onMouse)```
    - ```onMouse(event)``` : 마우스 이벤트를 처리하는 콜백 함수, 첫번째 인자로 이벤트를 받아줌

In [ ]:
img = cv.imread(DOG_PATH)

name = "Mouse Event"
cv.namedWindow(name)

def on_mouse(event, x, y, flags, _):
    if event == cv.EVENT_LBUTTONDOWN:
        copied = img.copy()
        cv.circle(copied, (x,y), 20, (255,255,0), cv.FILLED)
        cv.imshow(name, copied)

cv.setMouseCallback(name, on_mouse)

cv.imshow(name, img)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

In [ ]:
PRACTICE_PATH = "../images/book2.jpg"

img = cv.imread(PRACTICE_PATH)

name = "Mouse Event(Practice)"
point_list = []

def show_result():
    width = int(np.linalg.norm(np.array(point_list[0]) - (point_list[1])))
    height = int(np.linalg.norm(np.array(point_list[0]) - (point_list[3])))

    src = np.array(point_list, dtype=np.float32)
    dst = np.array([[0,0], [width,0], [width,height], [0,height]], dtype=np.float32)
    mat = cv.getPerspectiveTransform(src, dst)
    result = cv.warpPerspective(img, mat, (width, height))
    
    cv.imshow("warped", result)

drawing = False

def on_mouse(event, x, y, flags, _):
    global drawing
    coppied = img.copy()
    point = (x,y)

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        point_list.append(point)

    if drawing:
        prev_point = None
        for p in point_list:
            cv.circle(coppied, p, 10, (0,0,255), cv.FILLED)
            if prev_point:
                cv.line(coppied, prev_point, p, (0,0,255), 5)
            prev_point = p

        if len(point_list) == 4:
            point = point_list[0]
            show_result()
        cv.line(coppied, point_list[-1], point, (0,0,255), 5)

    cv.imshow(name, coppied)

cv.namedWindow(name)

cv.setMouseCallback(name, on_mouse)
cv.imshow(name, img)

cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(1)

-1

In [ ]:
PRACTICE_PATH = "../images/book2.jpg"

img = cv.imread(PRACTICE_PATH)

point_list = []

name = "Mouse Event"
cv.namedWindow(name)

def show_img():
    width = int(np.linalg.norm(np.array(point_list[0]) - (point_list[1])))
    height = int(np.linalg.norm(np.array(point_list[0]) - (point_list[3])))

    src = np.array(point_list, dtype=np.float32)
    dst = np.array([[0,0], [width,0], [width,height], [0,height]], dtype=np.float32)
    mat = cv.getPerspectiveTransform(src, dst)
    result = cv.warpPerspective(img, mat, (width, height))
    
    cv.imshow("result", result)

def on_mouse(event, x, y, flags, _):
    coppied = img.copy()
    point = (x,y)

    if event == cv.EVENT_LBUTTONDOWN:
        point_list.append(point)

    for i, p in enumerate(point_list):
        cv.circle(coppied, p, 10, (0,0,255), cv.FILLED)
        if i > 0:
            cv.line(coppied, point_list[i-1], p, (0,0,255), 5)

    if len(point_list) > 0 and len(point_list) < 4:
        cv.line(coppied, point_list[-1], point, (0,0,255), 5) 

    if len(point_list) == 4:
        cv.line(coppied, point_list[-1], point_list[0], (0,0,255), 5) 
        show_img()
    
    cv.imshow(name, coppied)

cv.setMouseCallback(name, on_mouse)

cv.imshow(name, img)

cv.waitKey(0)
cv.destroyAllWindows()
# cv.waitKey(1)